# Triples (or higher order) systems in each samples

In [1]:
import sys
sys.path.append('../src')
import P_binary
import P_random
import P_posterior
import const as c
import time
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import corner
import pickle


%matplotlib inline

/Library/Python/2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


### First, NLTT matches

In [16]:
# Full NLTT Catalog
filename = ('../data/rNLTT/catalog_tabs.dat')
t_full = Table.read(filename, format='ascii', guess=True)

# Change proper motion units from asec/yr to mas/yr
t_full['mu_ra'] = 1.0e3*t_full['mu_ra']
t_full['mu_dec'] = 1.0e3*t_full['mu_dec']
t_full['mu_ra_err'] = 1.0e3*t_full['mu_ra_err']
t_full['mu_dec_err'] = 1.0e3*t_full['mu_dec_err']

# Select only stars with proper motion uncertainties greater than 0.1 mas/yr - remove junk
ids_good = np.intersect1d(np.where(t_full['mu_ra_err'] >= 0.1), np.where(t_full['mu_dec_err'] >= 0.1))
t = t_full[ids_good]
print t.dtype

[('NLTT', '<i8'), ('ra', '<f8'), ('dec', '<f8'), ('mu_ra', '<f8'), ('mu_dec', '<f8'), ('mu_ra_err', '<f8'), ('mu_dec_err', '<f8'), ('V', '<f8'), ('B', '<f8'), ('R', '<f8'), ('J', '<f8'), ('H', '<f8'), ('K', '<f8')]


In [5]:
# Now read in matched pairs
NLTT_prob = pickle.load( open("../data/rNLTT/prob_out.data", "rb"))
print NLTT_prob.dtype

[('i_1', '<i4'), ('i_2', '<i4'), ('ID_1', '<i4'), ('ID_2', '<i4'), ('P_random', '<f8'), ('P_binary', '<f8'), ('P_posterior', '<f8')]


In [90]:
def find_triples(catalog):
    
    triple_pairs = match_triple_pairs(catalog)
    
    print ""
    print triple_pairs
    
def match_triple_pairs(catalog):

    dtype = ([('ID_1', 'i8'), ('ID_2', 'i8'), ('theta_1', 'f8')])
    triple_pairs = np.array([], dtype=dtype)
    
    N_triple = 0
    for i in np.arange(len(catalog)-1):
        
        i2 = np.arange(len(catalog) - i - 1) + i + 1
        for j in i2:
            if catalog['ID_1'][j] == catalog['ID_1'][i] \
               or catalog['ID_1'][j] == catalog['ID_2'][i] \
               or catalog['ID_2'][j] == catalog['ID_1'][i] \
               or catalog['ID_2'][j] == catalog['ID_2'][i]:
                    
                    
                # Default is to add the entry
                match_1 = True
                match_2 = True

                
                if triple_pairs.size != 0:
                    
                    # Add entry for first match
                    for k in np.arange(len(triple_pairs)):
                        ra1_a = t['ra'][np.where(t['NLTT'] == triple_pairs['ID_1'][k])][0]
                        ra2_a = t['ra'][np.where(t['NLTT'] == triple_pairs['ID_2'][k])][0]
                        ra1_b = t['ra'][np.where(t['NLTT'] == catalog['ID_1'][i])][0]
                        ra2_b = t['ra'][np.where(t['NLTT'] == catalog['ID_2'][i])][0]
                        if ra1_a == ra1_b and ra2_a == ra2_b: match_1 = False



                    # Add entry for second match
                    for k in np.arange(len(triple_pairs)):
                        ra1_a = t['ra'][np.where(t['NLTT'] == triple_pairs['ID_1'][k])][0]
                        ra2_a = t['ra'][np.where(t['NLTT'] == triple_pairs['ID_2'][k])][0]
                        ra1_b = t['ra'][np.where(t['NLTT'] == catalog['ID_1'][j])][0]
                        ra2_b = t['ra'][np.where(t['NLTT'] == catalog['ID_2'][j])][0]
                        if ra1_a == ra1_b and ra2_a == ra2_b: match_2 = False



                # Add entry for first match
                if match_1 == True:
                    id1 = np.where(t['NLTT'] == catalog['ID_1'][i])[0]
                    id2 = np.where(t['NLTT'] == catalog['ID_2'][i])[0]
                    theta = P_random.get_theta_proj_degree(t['ra'][id1], t['dec'][id1], t['ra'][id2], t['dec'][id2])[0]
                    print id1, id2, catalog['ID_1'][i], catalog['ID_1'][j], theta
                    match = np.zeros(1, dtype=dtype)
                    match[0] = catalog['ID_1'][i], catalog['ID_2'][i], theta
                    triple_pairs = np.append(triple_pairs, match)
                    
                # Add entry for second match
                if match_2 == True:
                    id1 = np.where(t['NLTT'] == catalog['ID_1'][j])[0][0]
                    id2 = np.where(t['NLTT'] == catalog['ID_2'][j])[0][0]
                    theta = P_random.get_theta_proj_degree(t['ra'][id1], t['dec'][id1], t['ra'][id2], t['dec'][id2])
                    match = np.zeros(1, dtype=dtype)
                    match[0] = catalog['ID_1'][j], catalog['ID_2'][j], theta
                    triple_pairs = np.append(triple_pairs, match)




    return triple_pairs

In [91]:
find_triples(NLTT_prob)

[2231] [2233] 3478 3478 0.00720520235116
[4395] [4399 4400] 6986 6986 0.015684150255
[5491] [5492] 8632 8632 0.0138785107953
[5559] [5562 5563] 8744 8744 0.000122189732312
[5901] [5903 5904] 9303 9303 0.0122428407409
[11129] [11130] 17264 17264 0.00352210320707
[12237 12238] [12237 12238] 18987 18987 0.0
[16890] [16891 16892] 26193 26193 0.00319316090387
[18458] [18460 18461] 28714 28714 0.00677687969047
[18875] [18877] 29410 29410 0.00958118807652
[22329] [22332 22333] 34713 34713 0.0156039305055
[26268 26269 26270] [26268 26269 26270] 41899 41899 0.0
[29237] [29238] 47260 47260 0.0
[29372] [29373 29374] 47506 47506 0.0012534889978
[30956 30957] [30956 30957] 50272 50272 0.0
[33545] [33551 33552] 54896 54896 0.0240053521383
[33762 33763] [33762 33763] 55240 55240 0.0

[(3478, 3481, 0.007205202351163697) (3478, 3482, 0.007205202351163697)
 (3481, 3482, 0.0) (6986, 6991, 0.01568415025504844)
 (6986, 6991, 0.01568415025504844) (6991, 6991, 0.0)
 (8632, 8633, 0.013878510795290276) (8632, 